# Create a custom Task

In this tutorial, we will go over `Task` objects, how they work and how to build a custom subclass to implement your own task design.

Let's start by importing what we need.


In [1]:
import os
import sys
import numpy as np
import tensorflow as tf

colab_env = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
colab_initialized = True if os.path.exists("MotorNet") else False

if colab_env and not colab_initialized:
  !git clone https://github.com/OlivierCodol/MotorNet
  sys.path.append('MotorNet')
  print("Running cell using COLAB initialization...")
elif colab_env and colab_initialized:
  print("Already initialized using COLAB initialization.")
else:
  paths = [p for p in sys.path if os.path.exists(p)]
  local_initialized = True if [p for p in paths if "motornet" in os.listdir(p)] else False
  if local_initialized:
    %load_ext autoreload
    %autoreload 2
    print("Already initialized using LOCAL initialization.")
  else:
    path = [p for p in paths if p.__contains__("tutorials")]
    if len(path) != 1:
      raise ValueError("Path to MotorNet could not be determined with certainty.")
    sys.path.append(os.path.dirname(path[:path.rfind('tutorials')]))
    %load_ext autoreload
    %autoreload 2
    print("Running cell using LOCAL initialization...")


import motornet as mn

print('All packages imported.')
print('tensorflow version: ' + tf.__version__)
print('numpy version: ' + np.__version__)

Already initialized using LOCAL initialization.
All packages imported.
tensorflow version: 2.9.1
numpy version: 1.22.4



# I. Useful methods in Task objects

Several methods are useful to assess what your task object currently contains.
- The `print_attributes` method will print all attributes held by the `Task` instance as well as their current value. This includes the losses declared via the `Task` object.
- The `get_attributes` method will fetch those attributes, and return two lists: one with the name of each attribute, and one with the associated value of each attribute.
- The `print_loss` method will print the losses currently declared via the `add_loss` method.

First, let's import a built-in `Task` object and create an instance of it.

In [2]:

plant = mn.plants.ReluPointMass24()
network = mn.nets.layers.GRUNetwork(plant=plant, n_units=50, kernel_regularizer=10**-6, name='network')

task = mn.tasks.CentreOutReach(network=network)


In [3]:

task.print_attributes()

angular_step:  15
catch_trial_perc:  50
delay_range:  [5, 25]
do_recompute_targets:  False
dt:  0.01
go_cue_range:  [5, 25]
initial_joint_state:  None
initial_joint_state_original:  None
n_initial_joint_states:  None
reaching_distance:  0.1
start_position:  [0.0, 0.0]
training_batch_size:  32
training_n_timesteps:  100

loss_weights:
 {'joint position': 0.0, 'cartesian position': 1.0, 'muscle state': 5, 'geometry state': 0.0, 'proprioceptive feedback': 0.0, 'visual feedback': 0.0, 'excitation': 0.0, 'gru_hidden_0': 0.1}

losses:
 {'joint position': None, 'cartesian position': <motornet.nets.losses.PositionLoss object at 0x000001886CB16A00>, 'muscle state': <motornet.nets.losses.L2xDxActivationLoss object at 0x000001886CB16970>, 'geometry state': None, 'proprioceptive feedback': None, 'visual feedback': None, 'excitation': None, 'gru_hidden_0': <motornet.nets.losses.L2xDxRegularizer object at 0x000001886CB16910>}

loss_names:
 {'joint position': 'joint position', 'cartesian position': '

In [4]:
attr_names, attr_values = task.get_attributes()
print(attr_names)
print(attr_values)

['angular_step', 'catch_trial_perc', 'delay_range', 'do_recompute_targets', 'dt', 'go_cue_range', 'initial_joint_state', 'initial_joint_state_original', 'n_initial_joint_states', 'reaching_distance', 'start_position', 'training_batch_size', 'training_n_timesteps']
[15, 50, [5, 25], False, 0.01, [5, 25], None, None, None, 0.1, [0.0, 0.0], 32, 100]


In [5]:

task.print_losses()



ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x000001886CB16A00>
loss weight:    1.0
loss name:      position
Compounded:     NO


ASSIGNED OUTPUT: muscle state
-----------------------------
loss function:  <motornet.nets.losses.L2xDxActivationLoss object at 0x000001886CB16970>
loss weight:    5
loss name:      l2_xdx_activation
Compounded:     NO


ASSIGNED OUTPUT: gru_hidden_0
-----------------------------
loss function:  <motornet.nets.losses.L2xDxRegularizer object at 0x000001886CB16910>
loss weight:    0.1
loss name:      gru_regularizer
Compounded:     NO





If we then add another loss using the `add_loss` method, and re-print the losses, the new loss will be included in the print-out.

Note that "Compounded" indicates if the loss shares an assigned output with another loss (see the tutorial on losses for more details).


In [6]:
task.add_loss(loss=mn.nets.losses.PositionLoss(), assigned_output="cartesian position")
task.print_losses()


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x000001886CB16A00>
loss weight:    1.0
loss name:      position
Compounded:     YES


ASSIGNED OUTPUT: cartesian position
-----------------------------------
loss function:  <motornet.nets.losses.PositionLoss object at 0x000001886CB5D0A0>
loss weight:    1.0
loss name:      position
Compounded:     YES


ASSIGNED OUTPUT: muscle state
-----------------------------
loss function:  <motornet.nets.losses.L2xDxActivationLoss object at 0x000001886CB16970>
loss weight:    5
loss name:      l2_xdx_activation
Compounded:     NO


ASSIGNED OUTPUT: gru_hidden_0
-----------------------------
loss function:  <motornet.nets.losses.L2xDxRegularizer object at 0x000001886CB16910>
loss weight:    0.1
loss name:      gru_regularizer
Compounded:     NO




# II. Initializing a MotorNetModel with a Task object


In `tensorflow`, creating a model requires declaring `tf.keras.layers.Input` layers that will handle the input and initial state data. Creating these `Input` layers mainly require to declare the shape of the arrays that will be passed through. The `Task` class should provide those using the `get_input_dict_layers` and `get_initial_state_layers` methods.



In [7]:
inputs = task.get_input_dict_layers()

for k, v in inputs.items():
    print(k + " :--> ", v)

inputs :-->  KerasTensor(type_spec=TensorSpec(shape=(None, None, 5), dtype=tf.float32, name='inputs'), name='inputs', description="created by layer 'inputs'")



As we can see, the input layer is simply a dictionary containing `keras` tensors.


In [8]:

state0 = task.get_initial_state_layers()

for s in state0:
    print(s)

KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='joint0'), name='joint0', description="created by layer 'joint0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='cartesian0'), name='cartesian0', description="created by layer 'cartesian0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4), dtype=tf.float32, name='muscle0'), name='muscle0', description="created by layer 'muscle0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4), dtype=tf.float32, name='geometry0'), name='geometry0', description="created by layer 'geometry0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 8, 1), dtype=tf.float32, name='proprio_feedback0'), name='proprio_feedback0', description="created by layer 'proprio_feedback0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2, 1), dtype=tf.float32, name='visual_feedback0'), name='visual_feedback0', description="created by layer 'visual_feedback0'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.fl

Conversely, the initial state is a list of `keras` tensors. This difference is simply because `tensorflow` models are built with that logic, and so require initial states to be lists. Inputs don't have to be dictionary but dictionaries are in practice clearer and more versatile than lists so we chose this over other possibilities.

# III. Subclassing a Task object

Now let's try to build our own task design. To do so, we will go over a simple subclassing process to create a custom task. The task we will try to build is a reaching task from a random starting position in the full joint space to a random target in that joint space. The movement should start from the first timestep of the simulation, and background loads will be constantly applied at the effector's endpoint.


### III. 1. Initialization of the Task subclass
The base class for tasks is `mn.tasks.Task`, so we will make the custom task inherit from that base class. We can then define a `__name__` for the class, and declare the losses in the `__init__` method using a `self.add_loss` method that is inherited from the base class.

Note that declaring losses through the `Task` object is not mandatory for the simulation to run, but this is easier, as otherwise we would have to manually do all the things that the `add_loss` method does automatically for us. Also this may result in misleading (shuffled) loss labels when printing progress bars at runtime because `tensorflow` models (`tf.keras.Model`) do not maintain loss label orders properly (the loss dictionary gets flattened). Adding losses to the task object will make it available to our curstom-made `tf.keras.Model` subclass, which is `mn.nets.MotorNetModel`. In `MotorNetModel` the losses available in `Task` subclasses will be re-ordered properly to avoid the parent class `tf.keras.Model` reshuffling the labels in a wrong order.

The `add_loss` method takes three arguments:
- `assigned_output` should be a string matching one of the keys of the `MotorNetModel`'s output dictionary. Of note, each key corresponds to a state of the model. In its current version, `motornet` only allows one loss per output key.
- `loss` should pass a loss object, either custom-made or from the `mn.nets.losses` module.
- `loss_weight` is a scalar that defines the weight of the loss in the global loss that the network will optimize at runtime.


### III. 2. Generating inputs
We can next define the `task.generate` method. It should take as inputs:
- `batch_size`
- `n_timesteps`

If desired, it can also take `**kwargs` inputs.

It should produce as output a list containing three items, in the order below:
- A dictionary containing the inputs to the network.
- The targets that will be passed to the network (often called `y` or `y_true`, as opposed to `x` or `y_pred`).
- The initial states to the network.


#### III. 2. a. Input dictionary
The input dictionary only requires a "inputs" key. The value assigned to that key will be passed as-is as input to the network for a forward pass. So is content is essentially up to the user, according to what the user wishes the network to receive as input information. Typically, for a reaching movement, this could be the target's position in cartesian coordinates. If a delayed reach is desired, one could consider adding a go cue as well.

Similar to states, the first dimension (rows) should always be  `batch_size` and the second dimension (columns) should always be the number of timesteps for the reach. This is true for any value in that dictionary, not just for the value associated with the "inputs" key.

Other notable keys one may add are "joint_load" and "endpoint_load". These would be transmitted to the plant by the network. If one wishes to add more keys to the input dictionary, it would be required to subclass the network to implement appropriate handling of these custom-made keys in the `call` method of the network.

We decided to use a dictionary as input because of the flexibility and clarity of code it provides. Note however that generally, `tensorflow` models can accept other forms of inputs, such as simple arrays, though our `motornet` model will not handle them.


#### III. 2. b. Targets
The target values that the plant should produce, and that will be passed to the loss functions. In `tensorflow` nomenclature, this is sometimes referred to as the `y_true`, to which the `y_pred` is compared.
In practice this could be the position of the target for a reaching movement. If a delayed movement is desired, it could be the starting position until the go cue time, and the target position afterwards.


#### III. 2. c. Initial states
The value of the initial states at simulation start. These can be obtained from the `task.get_initial_state(batch_size=batch_size)` method. If a pre-defined starting position is desired, one can pass an optional `joint_state` argument to this method as well.




In [9]:

class RandomTargetReachWithLoads(mn.tasks.Task):
    def __init__(self, network, endpoint_load: float, **kwargs):
        super().__init__(network, **kwargs)
        self.__name__ = 'RandomTargetReachWithLoads'
        self.endpoint_load = endpoint_load

        # losses
        max_iso_force = self.network.plant.muscle.max_iso_force
        c_loss = mn.nets.losses.PositionLoss()
        m_loss = mn.nets.losses.L2ActivationLoss(max_iso_force=max_iso_force)
        self.add_loss(assigned_output='cartesian position', loss=c_loss, loss_weight=1.)
        self.add_loss(assigned_output='muscle state', loss=m_loss, loss_weight=.2)

    def generate(self, batch_size, n_timesteps, **kwargs):
        validation = kwargs.get("validation", False)

        if not validation:
            # get random initial states (the default if no joint_state is provided)
            init_states = self.get_initial_state(batch_size=batch_size)
        else:
            # if validation, then always start in the center (0, 0)
            joint_state = tf.zeros((batch_size, self.network.plant.space_dim))
            init_states = self.get_initial_state(batch_size=batch_size, joint_state=joint_state)

        # random targets in joint coordinates
        goal_states_j = self.network.plant.draw_random_uniform_states(batch_size=batch_size)
        # make into cartesian coordinates (humans don't reach to targets specified in joint angles)
        goal_states = self.network.plant.joint2cartesian(goal_states_j)
        # this function pads the goal by n_timesteps: (batch_size, space_dim*2) -> (batch_size, n_timesteps, space_dim*2)
        # we make it from a tensor to a numpy array as well (best practice usually for the target)
        targets = self.network.plant.state2target(state=goal_states, n_timesteps=n_timesteps).numpy()

        inputs = {
          "inputs": targets[:, :, :self.network.plant.space_dim],  # network will know target position only, not velocity
          "endpoint_load": tf.constant(self.endpoint_load, shape=(batch_size, n_timesteps, 2))
        }
        return [inputs, targets, init_states]


task = RandomTargetReachWithLoads(network=network, endpoint_load=3.)
print("Task subclass built.\n")



L = task.generate(batch_size=5, n_timesteps=10, validation=True)

print("Dictionary content:")
for k, v in L[0].items():
    print("\t" + k + " shape :--> ", v.shape)

print("\ntargets shape :--> ", L[1].shape, "\n")

print("initial states shape:")
for elem in L[2]:
    print("\t", elem.shape)

Task subclass built.

Dictionary content:
	inputs shape :-->  (5, 10, 2)
	endpoint_load shape :-->  (5, 10, 2)

targets shape :-->  (5, 10, 4) 

initial states shape:
	 (5, 4)
	 (5, 4)
	 (5, 4, 4)
	 (5, 4, 4)
	 (5, 8, 1)
	 (5, 2, 1)
	 (5, 4)
	 (5, 50)
